In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import os
import sys
import time

import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import *
from metadata import *
from data_manager import *
from learning_utilities import *

/usr/local/lib/python2.7/dist-packages/matplotlib/__init__.py:1401: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


Setting environment for AWS compute node


No vtk
File does not exist: /shared/CSHL_data_processed/MD635/MD635_anchor.txt
File does not exist: /shared/CSHL_data_processed/MD635/MD635_sorted_filenames.txt
File does not exist: /shared/CSHL_data_processed/MD635/MD635_cropbox.txt
File does not exist: /shared/CSHL_data_processed/MD635/MD635_cropbox.txt


In [3]:
dataset_settings

,network_model,stain,margins,num_sample_per_class,stacks
dataset_id,,,,,
20,Inception-BN,nissl,200/500,1000,MD585
21,Inception-BN,nissl,200/500,1000,MD589
22,Inception-BN,nissl,200/500,1000,MD594
99,cell,nissl,500,1000,MD589


In [4]:
dataset_id = 99
dataset_properties = dataset_settings.loc[dataset_id]

num_samples_per_label = dataset_properties['num_sample_per_class']
stacks = dataset_properties['stacks'].split('/')

In [5]:
structures_to_sample = all_known_structures

negative_labels_to_sample = [s + '_negative' for s in structures_to_sample]

margins_to_sample = map(int, str(dataset_properties['margins']).split('/'))

surround_positive_labels_to_sample = [convert_to_surround_name(s, margin=m, suffix=surr_l) 
                             for m in margins_to_sample
                             for s in structures_to_sample 
                             for surr_l in structures_to_sample
                             if surr_l != s]

surround_noclass_labels_to_sample = [convert_to_surround_name(s, margin=m, suffix='noclass') 
                             for m in margins_to_sample
                             for s in structures_to_sample]

labels_to_sample = structures_to_sample + negative_labels_to_sample + \
surround_positive_labels_to_sample + surround_noclass_labels_to_sample + \
['noclass']

In [ ]:
def generate_dataset_cell_based(num_samples_per_label, stacks, labels_to_sample):
    """
    Generate dataset.
    - Extract addresses
    - Map addresses to features
    - Remove None features
    
    Returns:
        test_features, test_addresses, labels_found
    """
    
    # Extract addresses
    
    test_addresses = defaultdict(list)

    labels_found = set([])

    t = time.time()
    
    for stack in stacks:
        
        t1 = time.time()
        annotation_grid_indices_fn = os.path.join(ANNOTATION_ROOTDIR, stack, stack + '_annotation_grid_indices.h5')
        grid_indices_per_label = read_hdf(annotation_grid_indices_fn, 'grid_indices')
        sys.stderr.write('Read: %.2f seconds\n' % (time.time() - t1))

        labels_this_stack = set(grid_indices_per_label.index) & set(labels_to_sample)
        labels_found = labels_found | labels_this_stack

        t1 = time.time()
        test_addresses_sec_idx = sample_locations(grid_indices_per_label, labels_this_stack, 
                                                  num_samples_per_landmark=num_samples_per_label/len(stacks))
        sys.stderr.write('Sample: %.2f seconds\n' % (time.time() - t1))

        for label, addresses in test_addresses_sec_idx.iteritems():
            test_addresses[label] += [(stack, ) + addr for addr in addresses]

    test_addresses.default_factory = None
    
    sys.stderr.write('Sample addresses: %.2f seconds\n' % (time.time() - t))
    
    # Map addresses to features
    
    t = time.time()
    # test_features = apply_function_to_dict(lambda x: addresses_to_features(x, model_name=model_name), test_addresses)
    test_features = apply_function_to_dict(lambda x: addresses_to_features_parallel(x, model_name=model_name, n_processes=4), test_addresses)
    sys.stderr.write('Map addresses to features: %.2f seconds\n' % (time.time() - t))
    
    # Remove features that are None

    for name in labels_found:
        valid = [(ftr, addr) for ftr, addr in zip(test_features[name], test_addresses[name])
                    if ftr is not None]
        res = zip(*valid)
        test_features[name] = np.array(res[0])
        test_addresses[name] = res[1]
        
    return test_features, test_addresses, labels_found

In [ ]:
features, addresses, labels_found = generate_dataset(num_samples_per_label=num_samples_per_label, 
                                                     stacks=stacks,
                                                     labels_to_sample=labels_to_sample)